## Section 1. Introduction ##

In this notebook, the dataset to be processed is the Labor Force Survey conducted April 2016 and retrieved through Philippine Statistics Authority database. 



In [ ]:
import random
import numpy as np
import pickle
import os
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (6.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.style.use('ggplot')

# autoreload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

<h1>Importing LFS PUF April 2016.CSV</h1>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import plot_tree
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

try:
    lfs_data = pd.read_csv("LFS PUF April 2016.CSV")
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory or provide the correct path.")
    exit()


<h1>Data Information, Pre-Processing, and Cleaning</h1>

Let's get an overview of our dataset.

In [ ]:
lfs_data.info()

---
Of interest to us, there are:
<ul><li>1 contains float values, </li>
<li>14 contain integer values, and </li>
<li><b>35 are object values</b>.</li></ul>

Let's check for duplicates:

In [ ]:
lfs_data.duplicated().sum()

No duplicates here, and therefore no cleaning need follow in this regard.

The dataset seems to contain null values in the form of whitespaces. Let's count those:

In [ ]:
has_null = lfs_data.apply(lambda col: col.str.isspace().sum() if col.dtype == 'object' else 0)

print("Number Empty Cells:")
print(has_null[has_null > 0])

---
And standardize, replacing these whitespace values with NaN:

In [ ]:
lfs_data.replace(r"^\s+$", np.nan, regex=True, inplace=True)
nan_counts_per_column = lfs_data.isna().sum()
print(nan_counts_per_column[nan_counts_per_column > 0])

---
Let's also apply the unique() function to our dataset.

In [ ]:
lfs_data.apply(lambda x: x.nunique())

---
Considering our dataset has 18,000 entries, features with particularly low numbers stand out as questions that have clear, defined choices. Reviewing the [questionnaire](https://psada.psa.gov.ph/catalog/67/download/537), we find that certain questions ask the participant to specify beyond prespecified choices.

This column possibly contains "010," which is obviously not an integer. We ensure this column is a string, and check for values not specified in the questionnaire.

In [ ]:
lfs_data['PUFC07_GRADE'] = lfs_data['PUFC07_GRADE'].astype(str)
valid_codes = [
    "000", "010",                                      # No Grade, Preschool
    "210", "220", "230", "240", "250", "260", "280",  # Elementary
    "310", "320", "330", "340", "350",                # High School
    "410", "420",                                     # Post Secondary; If Graduate Specify
    "810", "820", "830", "840",                       # College; If Graduate Specify
    "900",                                            # Post Baccalaureate
    "nan"
]
invalid_rows = lfs_data[~(lfs_data['PUFC07_GRADE'].isin(valid_codes))]

unique_invalid_values = invalid_rows['PUFC07_GRADE'].unique()
print(unique_invalid_values)

Values 5XX 6XX are not detailed in the questionnaire. As it instructs the participant to specify whether they graduated from post secondary or college, we'll create a new data point to encapsulate these.

In [ ]:
lfs_data.loc[~lfs_data['PUFC07_GRADE'].isin(valid_codes), 'PUFC07_GRADE'] = '700'
print(lfs_data['PUFC07_GRADE'].unique())

Convert all columns to string first to avoid issues with LabelEncoder

In [ ]:
lfs_data=lfs_data.astype(str)

Use LabelEncoder to convert categorical variables to integer

In [ ]:
encoder = LabelEncoder()
for col in ['PUFC09_GRADTECH', 'PUFC07_GRADE', 'PUFC08_CURSCH']:
    lfs_data[col] = encoder.fit_transform(lfs_data[col])



Split the data into predictors (x) and target (y)

In [ ]:
X = lfs_data[['PUFC09_GRADTECH', 'PUFC07_GRADE', 'PUFC05_AGE']]
y = lfs_data['PUFC08_CURSCH']

Splits the dataset into 80% training and 20% testing sets.
random state is the seed for the random integer generator

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
model = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=69)
model.fit(X_train, y_train)

train 🚂

In [ ]:
model = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=420)
model.fit(X_train, y_train)

test

In [ ]:
y_pred = model.predict(X_test)


eval

In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))